## Exercise 1 Representations and their operators for crossover and mutation

__Binary:__ 
<br>
On the computer. Spices example. Each gene represents one spice, and the value can be true or false.
<br>

Mutation:<br> 
Bit-flip. With a probability $p_m$ flip one, or several, digits.
<br>

Recombination:<br>  
L#2 s. 32-: Single-point crossover, n-point crossover, uniform crossover (Tail = parent 1, heads = parent 2, flip through each gene. Child 2 opposite of child 1.)
<br>

__Integer:__ 
<br>
TSP problem, 8 queens chess problem. Vector of different integers, <mark> DIP, colors (red, blue)
<br>
Mutation: <br>
Creeping (slow) or resetting (larger change)
<br>
Crossover:<br> 
    N-point, uniform.

__Floating point numbers:__ 
<br>
Maximization of numbers smaller than integers.
<br>
Mutation: Unform mutation, same role as bit flip for bindary or random resetting for integers. Draw from [LB, UB], upper lower bound. 
<br>
Non-uniform mutation: Use gaussian each variable seprately.
<br>
Crossover: <br>
Discrete recombination: Value always from 1 of parents (No averages). n-point crossover or uniform.  
<br>
Intermediate recombination: Children, weighted average of parents, as I did in my 1st EA.
<br>
Example: Simple arithmetic crossover: Crossover + intermediate after crossover point.
<br>
Example: Single arithmetic crossover. Child 1 equal parent 1 to the left of point, at point a weighted average, and parent 2 to the right of point.
<br>
Example: Whole arithmetic crossover: weighted average everywhere.
<br>

__Trees:__ Loops, mathematical expressions.
<br>
Mutation: replace randomly chosen subtree by randomly generated tree
<br>
Crossover: Most common recombination: exchange two randomly chosen subtrees among the parents
<br>

__Permutation:__ 
<br>
Travelling sales person. One number only appears once. List of n different integers.
<br>
Mutation:
    <br>
    Must change at least 2 values. Swapping. Insert mutation: Move one next to another and change all others to come next in same order as they were. Scramble mutation: for certain number of genes randomly reorder these genes. Inversion: For a chosen area, reverse the order of the genes.
 <br>
Crossover:
<br>
   Adjacance important (who is neighbor). TSP. --> Conserve adjacancy.
   <br>
   Partially mapped crossover and edge recombination: L#2 s. 52
   <br>
   Order important others. Need to conserve order. Order corssover and cycle crossover.
   <br>
   Partially mapped crossover (PMX). See slides L#2 s 56.
   <br>
   Edge recombination. See slide.
   <br>
   Order crossover: L#2 s 61. Understandable example.
   <br>
   Cycle crossover: L#2 s 62. 


## Exercsise 2: Bit flip mutation

In [7]:
import scipy.special

pm = 1./4
p0 = (1 - pm)**4

n = 4
p1 = scipy.special.binom(n, 1)*pm**1*(1-pm)**(n-1)

pMoreThan1 = 0
for k in range(2,4+1):
    pMoreThan1 += scipy.special.binom(n, k)*pm**k*(1-pm)**(n-k)

print('p(0) = ', p0, '\n p(1) = ', p1, '\n p(x>1) = ', pMoreThan1)

p(0) =  0.31640625 
 p(1) =  0.421875 
 p(x>1) =  0.26171875


Altnernatively, $P(x>1) = P(x \geq 2) = 1 - P(0) - P(1)$

## Exercise 3: Crossover

In [347]:
class Crossover:
    ''' Crossover. Input = parent arrays, start and stop position of genes to keep from one parent.'''
    def __init__(self, p1, p2, startPosition, stopPosition):
        self.p1, self.p2, self.startPosition, self.stopPosition =\
        p1, p2, startPosition, stopPosition
        
    def pmx(self):
        p1, p2, startPosition, stopPosition = \
        self.p1, self.p2, self.startPosition, self.stopPosition
        
        c = np.zeros(len(p1))
        c[startPosition:stopPosition] = p1[startPosition:stopPosition]

        for position in np.arange(startPosition,stopPosition,1):
            if p2[position] not in c[startPosition:stopPosition]:
                number = p2[position]
                p2index = position
                while p2index in np.arange(startPosition,stopPosition,1):
                    cValue = c[p2index]
                    p2index = np.where( p2 == cValue)
                c[p2index] = number
        
        remainingPositions = np.where( c == 0)
        
        for position in remainingPositions:
            c[position] = p2[position]
        return c
    
    def orderCrossover(self):
        p1, p2, startPosition, stopPosition = \
        self.p1, self.p2, self.startPosition, self.stopPosition
        
        c = np.zeros(len(p1))
        c[startPosition:stopPosition] = p1[startPosition:stopPosition]
        
        indexArray = np.arange(len(c))
        remainingPositions = np.concatenate([indexArray[stopPosition:], indexArray[0:startPosition]])
        coveredPositions = np.arange(startPosition,stopPosition)
        p2TryIndices =  np.concatenate([remainingPositions, coveredPositions])
        
        newPositionNumber = 0
        for position in remainingPositions:
            tryIndex = newPositionNumber
            while p2[p2TryIndices[tryIndex]] in c:
                tryIndex += 1
            c[position] = p2[p2TryIndices[tryIndex]] 
            newPositionNumber +=1
            
        return c
    
    def cyclicCrossover(self):
        p1, p2, startPosition, stopPosition = \
        self.p1, self.p2, self.startPosition, self.stopPosition
        
        c = np.zeros(len(p1))
        
        row1Keep = []
        counter = 0
        row1Position = 0
        startCycling = True
        start = 0

        while startCycling:
            row2 = p2[start]
            while row2 != p1[start] and counter < len(p1)+1:
                startCycling = False
                if len(row1Keep) == 0:
                    row1Keep.append(np.array(([start])))
                row2 = p2[row1Position]
                row1Position = np.where( p1 == row2)
                row1Keep.append(np.concatenate(row1Position))
                row2 = p2[row1Position]
                counter += 1
            start += 1
        row1Keep = np.concatenate(row1Keep)
        
        c = p2
        c[row1Keep] = p1[row1Keep]
        
        return c

## PMX
When adjacency important. <br>

__Book example__

In [146]:
p1 = np.asarray(np.arange(1,9+1,1))
p2 = np.asarray((9,3,7,8,2,6,5,1,4))
print(p1)
print(p2)
startPosition = 3
stopPosition = 6+1
pmx = Crossover(p1, p2, startPosition, stopPosition)
child = pmx.pmx()
print(child)

[1 2 3 4 5 6 7 8 9]
[9 3 7 8 2 6 5 1 4]
[9. 3. 2. 4. 5. 6. 7. 1. 8.]


__Exercise values__

In [171]:
p1 = np.array((2,4,7,1,3,6,8,9,5))
p2 = np.array((5,9,8,6,2,4,1,3,7))
startPosition = 3
stopPosition = 6+1
pmx = Crossover(p1, p2, startPosition, stopPosition)
child = pmx.pmx()
print(child)

pmx2 = Crossover(p2, p1, startPosition, stopPosition)
child = pmx2.pmx()
print(child)

[5. 9. 4. 1. 3. 6. 8. 2. 7.]
[3. 8. 7. 6. 2. 4. 1. 9. 5.]


## Order crossover
When the relative order of the genes are important to conserve.<br>

__Book example__

In [285]:
p1 = np.array((1,2,3,4,5,6,7,8,9))
p2 = np.array((9,3,7,8,2,6,5,1,4))
startPosition = 3
stopPosition = 6+1
oc = Crossover(p1, p2, startPosition, stopPosition)
child = oc.orderCrossover()
print('\n cild',child)


 cild [3. 8. 2. 4. 5. 6. 7. 1. 9.]


__Exercise__

In [286]:
p1 = np.array((2,4,7,1,3,6,8,9,5))
p2 = np.array((5,9,8,6,2,4,1,3,7))
startPosition = 3
stopPosition = 6+1
oc = Crossover(p1, p2, startPosition, stopPosition)
child = oc.orderCrossover()
print('\n cild',child)


 cild [9. 2. 4. 1. 3. 6. 8. 7. 5.]


## Cycle crossover
Used when absolute position is important, meaning that conservation of exact positions of allele's is important.

__Book example__

In [348]:
p1 = np.array((1,2,3,4,5,6,7,8,9))
p2 = np.array((9,3,7,8,2,6,5,1,4))
startPosition = 3
stopPosition = 6+1
cc = Crossover(p1, p2, startPosition, stopPosition)
child = cc.cyclicCrossover()
print('\n cild',child)


 cild [1 3 7 4 2 6 5 8 9]


__Exercise__

In [349]:
p1 = np.array((2,4,7,1,3,6,8,9,5))
p2 = np.array((5,9,8,6,2,4,1,3,7))
startPosition = 3
stopPosition = 6+1
cc = Crossover(p1, p2, startPosition, stopPosition)
child = cc.cyclicCrossover()
print('\n cild',child)


 cild [2 4 7 1 3 6 8 9 5]
